In [39]:
import pandas as pd
import numpy as np
import datetime

In [40]:
# importando dados
df = pd.read_csv('data\gastos_presidenciais.csv')
df

,ano,presidente,data_pagamento,cpf_servidor,cpf_cnpj_fornecedor,nome_fornecedor,tipo,subelemento_despesa,cdic,valor
0,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1696.90
1,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1336.60
2,2003,Lula,2003-01-02,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,1396.43
3,2003,Lula,2003-01-03,***.004.131-**,00592717000170,FLORES ALVORADA,C,OUTROS SERVI€OS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,8585.00
4,2003,Lula,2003-01-03,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COM RCIO DE MATERIAIS PARA L...,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,251.50
...,...,...,...,...,...,...,...,...,...,...
113335,2022,Bolsonaro,2022-12-03,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,179.50
113336,2022,Bolsonaro,2022-12-03,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,54.95
113337,2022,Bolsonaro,2022-12-04,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUI€ÇO,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,518.07
113338,2022,Bolsonaro,2022-12-04,***.136.591-**,12550441000146,RESTAURANTE SOHO,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,330.52


In [41]:
# convertendo coluna para data
df['data_pagamento'] = pd.to_datetime(df['data_pagamento'])

In [42]:
# criando coluna com o mês
df['mes'] = pd.DatetimeIndex(df['data_pagamento'])
df['mes'] = df['data_pagamento'].dt.month_name()
df

,ano,presidente,data_pagamento,cpf_servidor,cpf_cnpj_fornecedor,nome_fornecedor,tipo,subelemento_despesa,cdic,valor,mes
0,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1696.90,January
1,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1336.60,January
2,2003,Lula,2003-01-02,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,1396.43,January
3,2003,Lula,2003-01-03,***.004.131-**,00592717000170,FLORES ALVORADA,C,OUTROS SERVI€OS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,8585.00,January
4,2003,Lula,2003-01-03,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COM RCIO DE MATERIAIS PARA L...,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,251.50,January
...,...,...,...,...,...,...,...,...,...,...,...
113335,2022,Bolsonaro,2022-12-03,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,179.50,December
113336,2022,Bolsonaro,2022-12-03,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,54.95,December
113337,2022,Bolsonaro,2022-12-04,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUI€ÇO,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,518.07,December
113338,2022,Bolsonaro,2022-12-04,***.136.591-**,12550441000146,RESTAURANTE SOHO,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,330.52,December


In [43]:
# criando coluna com mês abreviado e ano
df['mes'] = df['data_pagamento'].apply(lambda x: datetime.datetime.strftime(x, '%b'))
df['date'] = pd.to_datetime(df['mes'] + '/' + df['ano'].astype(str), format='%b/%Y')

In [44]:
# importando dados da inflação do IBGE e deletando colunas desnecessárias
infl = pd.read_excel('data\Tabela 1737.xlsx')
infl.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis = 1, inplace=True)
infl.columns

Index(['mes', 'ano', 'infl'], dtype='object')

In [45]:
# renomeando colunas e convertendo para data
infl.mes.replace(['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], inplace=True)
infl['date'] = pd.to_datetime(infl['mes'] + '/' + infl['ano'].astype(str), format='%b/%Y')

In [46]:
# separando apenas colunas necessárias e unindo com o df
infl2 = infl[['infl', 'date']]
df_merged = pd.merge(df, infl2, on = 'date')

In [47]:
# criando coluna com valor atualizado pela inflação
df_merged['valor_atualizado'] = (df_merged['infl'].iloc[-1] / df_merged['infl']) * df_merged['valor']
df_merged

,ano,presidente,data_pagamento,cpf_servidor,cpf_cnpj_fornecedor,nome_fornecedor,tipo,subelemento_despesa,cdic,valor,mes,date,infl,valor_atualizado
0,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1696.90,Jan,2003-01-01,2085.68,5267.290918
1,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1336.60,Jan,2003-01-01,2085.68,4148.895657
2,2003,Lula,2003-01-02,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,1396.43,Jan,2003-01-01,2085.68,4334.611972
3,2003,Lula,2003-01-03,***.004.131-**,00592717000170,FLORES ALVORADA,C,OUTROS SERVI€OS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,8585.00,Jan,2003-01-01,2085.68,26648.413299
4,2003,Lula,2003-01-03,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COM RCIO DE MATERIAIS PARA L...,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,251.50,Jan,2003-01-01,2085.68,780.672795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113335,2022,Bolsonaro,2022-12-03,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,179.50,Dec,2022-12-01,6474.09,179.500000
113336,2022,Bolsonaro,2022-12-03,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,54.95,Dec,2022-12-01,6474.09,54.950000
113337,2022,Bolsonaro,2022-12-04,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUI€ÇO,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,518.07,Dec,2022-12-01,6474.09,518.070000
113338,2022,Bolsonaro,2022-12-04,***.136.591-**,12550441000146,RESTAURANTE SOHO,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,330.52,Dec,2022-12-01,6474.09,330.520000


In [48]:
# agrupando por presidente e ano e somando os valores
gastos_por_presidente = df_merged.groupby(['presidente', 'ano'])['valor_atualizado'].sum().sort_values(ascending=False).reset_index()
gastos_por_presidente['valor_atualizado'] = gastos_por_presidente['valor_atualizado'].div(1000000).round(2)
gastos_por_presidente.sort_values(by=['ano'], ascending = True).reset_index(drop=True)

,presidente,ano,valor_atualizado
0,Lula,2003,15.84
1,Lula,2004,18.28
2,Lula,2005,13.56
3,Lula,2006,12.48
4,Lula,2007,9.34
5,Lula,2008,13.85
6,Lula,2009,11.88
7,Lula,2010,13.59
8,Dilma,2011,8.92
9,Dilma,2012,8.50


In [49]:
# resumo dos gastos por presidente
resumo_gastos_por_presidente = gastos_por_presidente.groupby(['presidente'])['valor_atualizado'].sum().reset_index().sort_values(by=['valor_atualizado'], ascending = False).reset_index(drop=True)
resumo_gastos_por_presidente

,presidente,valor_atualizado
0,Lula,108.82
1,Dilma,57.47
2,Bolsonaro,31.54
3,Temer,11.62


In [50]:
# número de anos que cada presidente esteve no poder
mandato = 4
anos_mandato_presidente = df_merged.groupby('presidente')['ano'].nunique().sort_values(ascending=True).reset_index()
anos_mandato_presidente['mandatos'] = anos_mandato_presidente['ano'] / mandato
anos_mandato_presidente

,presidente,ano,mandatos
0,Temer,2,0.5
1,Bolsonaro,4,1.0
2,Dilma,6,1.5
3,Lula,8,2.0


In [51]:
# valor médio gasto por presidente por mandato
valor_medio_gasto_por_mandato = pd.merge(resumo_gastos_por_presidente, anos_mandato_presidente, on = 'presidente')
valor_medio_gasto_por_mandato['valor_medio_gasto_por_mandato'] = valor_medio_gasto_por_mandato['valor_atualizado'] / valor_medio_gasto_por_mandato['mandatos']
valor_medio_gasto_por_mandato.sort_values(by=['valor_medio_gasto_por_mandato'], ascending = False).reset_index(drop=True)
valor_medio_gasto_por_mandato

,presidente,valor_atualizado,ano,mandatos,valor_medio_gasto_por_mandato
0,Lula,108.82,8,2.0,54.410000
1,Dilma,57.47,6,1.5,38.313333
2,Bolsonaro,31.54,4,1.0,31.540000
3,Temer,11.62,2,0.5,23.240000


In [62]:
# tipo de despesa por presidente
tipo_despesa_por_presidente = df_merged.groupby(['presidente', 'subelemento_despesa'])['valor_atualizado'].sum().div(1000000).round(2).sort_values(ascending=False).reset_index()
tipo_despesa_por_presidente_pivoted = pd.pivot_table(tipo_despesa_por_presidente, values = 'valor_atualizado', index = 'subelemento_despesa', columns = 'presidente', aggfunc = 'sum', fill_value = 0).reset_index()
tipo_despesa_por_presidente_pivoted['total'] = tipo_despesa_por_presidente_pivoted.sum(axis=1)

for presidente in tipo_despesa_por_presidente_pivoted.columns[1:5]:
    tipo_despesa_por_presidente_pivoted[presidente+'%'] = tipo_despesa_por_presidente_pivoted[presidente] / tipo_despesa_por_presidente_pivoted['total'] * 100
    
tipo_despesa_por_presidente_pivoted['total %'] = tipo_despesa_por_presidente_pivoted['total'] / total * 100 # needs correction
tipo_despesa_por_presidente_pivoted.sort_values(by='total', ascending=False).head(10)


C:\Users\felip\AppData\Local\Temp\ipykernel_11732\1637473221.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tipo_despesa_por_presidente_pivoted['total'] = tipo_despesa_por_presidente_pivoted.sum(axis=1)


presidente,subelemento_despesa,Bolsonaro,Dilma,Lula,Temer,total,Bolsonaro%,Dilma%,Lula%,Temer%,total %
32,HOSPEDAGENS,15.69,30.21,48.85,4.28,99.03,15.843684,30.505907,49.328486,4.321923,47.294522
35,LOCACAO DE MEIOS DE TRANSPORTE,0.04,0.19,30.49,0.00,30.72,0.130208,0.618490,99.251302,0.000000,14.671188
31,GENEROS DE ALIMENTA€ÇO,5.43,9.46,10.32,3.04,28.25,19.221239,33.486726,36.530973,10.761062,13.491571
28,FORNECIMENTO DE ALIMENTA€ÇO,6.26,6.96,8.69,1.69,23.60,26.525424,29.491525,36.822034,7.161017,11.270834
87,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",1.78,6.47,2.44,2.12,12.81,13.895394,50.507416,19.047619,16.549571,6.117771
98,SERVI€OS DE TELECOMUNICA€åES,0.02,0.59,1.77,0.01,2.39,0.836820,24.686192,74.058577,0.418410,1.141411
11,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,0.75,0.37,0.77,0.08,1.97,38.071066,18.781726,39.086294,4.060914,0.940828
59,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,0.23,0.70,0.88,0.08,1.89,12.169312,37.037037,46.560847,4.232804,0.902622
36,LOCA€ÇO BENS MOV. OUTR. NATUREZA E INTANGÖVEIS,0.77,0.25,0.58,0.00,1.60,48.125000,15.625000,36.250000,0.000000,0.764124
56,MATERIAL DE COPA E COZINHA,0.06,0.23,0.38,0.05,0.72,8.333333,31.944444,52.777778,6.944444,0.343856


In [54]:
# tipo de despesa mais frequente

tipo_despesa_mais_frequente = df_merged.groupby(['presidente', 'subelemento_despesa'])['valor_atualizado'].count().sort_values(ascending=False).reset_index()
tipo_despesa_mais_frequente_pivoted = pd.pivot_table(tipo_despesa_mais_frequente, values = 'valor_atualizado', index = 'subelemento_despesa', columns = 'presidente', aggfunc = 'sum', fill_value = 0).reset_index()
tipo_despesa_mais_frequente_pivoted['total'] = tipo_despesa_mais_frequente_pivoted.sum(axis=1)
tipo_despesa_mais_frequente_pivoted.sort_values(by='total', ascending=False).head(10)

C:\Users\felip\AppData\Local\Temp\ipykernel_11732\2617580782.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tipo_despesa_mais_frequente_pivoted['total'] = tipo_despesa_mais_frequente_pivoted.sum(axis=1)


presidente,subelemento_despesa,Bolsonaro,Dilma,Lula,Temer,total
32,HOSPEDAGENS,1281,5942,37648,419,45290
31,GENEROS DE ALIMENTA€ÇO,7449,7898,4460,5116,24923
28,FORNECIMENTO DE ALIMENTA€ÇO,958,1586,3660,394,6598
98,SERVI€OS DE TELECOMUNICA€åES,34,1818,4007,61,5920
87,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",835,1742,1631,674,4882
84,PEDAGIOS,120,1248,1493,172,3033
11,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,652,635,1380,328,2995
59,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,745,820,1131,118,2814
35,LOCACAO DE MEIOS DE TRANSPORTE,21,3,1986,0,2010
66,MATERIAL FARMACOLOGICO,221,1173,264,10,1668


In [55]:
# tipo de despesa mais frequente por oresudebte
tipo_despesa_mais_frequente = df_merged.groupby(['presidente', 'subelemento_despesa'])['valor_atualizado'].count().sort_values(ascending=False).reset_index()
tipo_despesa_mais_frequente.head(10)

,presidente,subelemento_despesa,valor_atualizado
0,Lula,HOSPEDAGENS,37648
1,Dilma,GENEROS DE ALIMENTA€ÇO,7898
2,Bolsonaro,GENEROS DE ALIMENTA€ÇO,7449
3,Dilma,HOSPEDAGENS,5942
4,Temer,GENEROS DE ALIMENTA€ÇO,5116
5,Lula,GENEROS DE ALIMENTA€ÇO,4460
6,Lula,SERVI€OS DE TELECOMUNICA€åES,4007
7,Lula,FORNECIMENTO DE ALIMENTA€ÇO,3660
8,Lula,LOCACAO DE MEIOS DE TRANSPORTE,1986
9,Dilma,SERVI€OS DE TELECOMUNICA€åES,1818
